In [2]:

from tqdm import tqdm
import pickle
import pandas as pd
from pathlib import Path
import numpy as np
import copy
import sys
import os
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import rc
from mpl_toolkits.axes_grid1 import make_axes_locatable
from sklearn.metrics.pairwise import cosine_similarity,cosine_distances

sys.path.append('/mnt/c/Users/lslat/Code/')
sys.path.append('/mnt/c/Users/lslat/QiskitProjects/VariationalWavefunction/')

from quantum_kernel.code.utils import get_dataset
from quantum_kernel.code.visualization_utils import aggregate_folder
from quantum_kernel.code.kernel_measures import kernel_geometry
from quantum_kernel.code.utils import compute_additional_fields

font = {'family': 'DejaVu Sans',
        'weight': 'bold',
        'size': 18}

rc('font', **font)

tqdm.pandas()

In [7]:
# Get kernel results for the different ml models and dataset.d
dfs = {}
dataset_name='fashion-mnist'
folders = [
    ("/mnt/c/Users/lslat/Data/QK_project/results/Sparse_IQPm/"+dataset_name,'Sparse_IQPm',False),
    ("/mnt/c/Users/lslat/Data/QK_project/results/hproj_Sparse_IQP/"+dataset_name,'Sparse_IQPm',True),

]

for folder, kernel_name,projected in folders:
    if dataset_name in dfs:
        temp = aggregate_folder(folder, dataset_name, kernel_name=kernel_name,projected=projected)[
            dataset_name]
        dfs[dataset_name] = pd.concat([dfs[dataset_name], temp])
    else:
        dfs[dataset_name] = aggregate_folder(folder, dataset_name, kernel_name=kernel_name,projected=projected)[dataset_name]

save_folder='/mnt/c/Users/lslat/Data/QK_project/figs/'


100%|██████████| 10/10 [03:08<00:00, 18.84s/it]


For /mnt/c/Users/lslat/Data/QK_project/results/hproj_Sparse_IQP/fashion-mnist, saving aggregated pickle in /mnt/c/Users/lslat/Data/QK_project/results/hproj_Sparse_IQP/fashion-mnist/aggregated.p


In [12]:
df = dfs[dataset_name]
iqpm_df = df[(df['kernel_name'] == 'Sparse_IQPm') & (df['dataset_dim'] == 18) & (df['projected']=='') & (df['log_int_scaling_factor']==0.0)]
proj_iqpm_df = df[(df['kernel_name'] == 'Sparse_IQPm') & (df['dataset_dim'] == 18) & (df['projected']=='huang_proj') & (df['log_int_scaling_factor']==0.0)]

In [37]:
proj_iqpm_df['train_matrix']=proj_iqpm_df.apply(lambda x: np.exp(-x['proj_gamma']*x['qkern_matrix_train']),axis=1)

0    [[1.0, 0.9429818145022381, 0.9641603991274619,...
1    [[1.0, 0.9765089891762007, 0.9854928091344793,...
2    [[1.0, 0.9293276400628461, 0.9542529266094677,...
3    [[1.0, 0.9967863098142177, 0.997807899227251, ...
4    [[1.0, 0.9148824864004336, 0.9319865190947659,...
5    [[1.0, 0.3307660550809476, 0.19548622109064487...
6    [[1.0, 0.7004609401710218, 0.681157495259793, ...
7    [[1.0, 0.6048104287459612, 0.5716059097952223,...
8    [[1.0, 0.33808396086770454, 0.4782822529616101...
9    [[1.0, 0.7420917400560543, 0.7525548547602615,...
dtype: object

In [ ]:
df1 = iqpm_df[['log_scaling_factor', 'qkern_matrix_train']]
df2 = proj_iqpm_df[['log_scaling_factor', 'train_matrix']]
# compute the metric matrix and the list of sorted hyperparameters
h1s, h2s, metric_matrix = kernel_geometry.compute_metric_matrix(df1, df2, metric='geometric_difference')
# compute the reverse
rh1s, rh2s, rmetric_matrix = kernel_geometry.compute_metric_matrix(df2, df1, metric='geometric_difference')

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(18, 18))
# Plot the geometric difference matrix
ax = axs[0][0]
img = ax.imshow(np.log(metric_matrix))

ax.xaxis.tick_top()
ax.set_xticks(np.arange(h1s.shape[0]), minor=False)
ax.set_xticklabels(h1s, rotation=45)
ax.set_xlabel(r'$\gamma_{rbf}$')
ax.xaxis.set_label_position('top')

ax.set_yticks(np.arange(h2s.shape[0]), minor=False)
ax.set_yticklabels(h2s)
ax.set_ylabel(r'$log(\lambda_{IQP})$')

divider = make_axes_locatable(ax)
cax = divider.append_axes('right', size='5%', pad=0.05)
fig.colorbar(img, cax=cax)
ax.set_title(r'$\log{g(K_{rbf}||K_{IQP})}$')
# Now plot the reverse geometric difference matrix
ax = axs[0][1]
img = ax.imshow(np.log(rmetric_matrix))

ax.xaxis.tick_top()
ax.set_xticks(np.arange(rh1s.shape[0]), minor=False)
ax.set_xticklabels(rh1s, rotation=45)
ax.set_xlabel(r'$log(\lambda_{IQP})$')
ax.xaxis.set_label_position('top')

ax.set_yticks(np.arange(rh2s.shape[0]), minor=False)
ax.set_yticklabels(rh2s)
ax.set_ylabel(r'$\gamma_{rbf}$')

divider = make_axes_locatable(ax)
cax = divider.append_axes('right', size='5%', pad=0.05)
fig.colorbar(img, cax=cax)
ax.set_title(r'$\log{g(K_{IQP}||K_{rbf})}$')
# Plot the test score as a function of gamma for the RBF kernel.
ax = axs[1][0]
x = h1s
y = np.array([iqpm_df['test_score'].loc[iqpm_df['gamma'] == h1].values[0]
             for h1 in h1s])
img = ax.plot(y)

ax.set_xticks(np.arange(h1s.shape[0]), minor=False)
ax.set_xticklabels(h1s, rotation=45)
ax.set_xlabel(r'$\gamma_{rbf}$')

ax.set_ylabel('Test score')

ax.set_title('RBF kernel test score')
# Plot the test score as a function of gamma for the Laplacian Kernel.
ax = axs[1][1]
x = h2s
y = np.array([proj_iqpm_df['test_score'].loc[proj_iqpm_df['log_scaling_factor']
             == h2].values[0] for h2 in h2s])
img = ax.plot(y)

ax.set_xticks(np.arange(h2s.shape[0]), minor=False)
ax.set_xticklabels(h2s, rotation=45)
ax.set_xlabel(r'$log(\lambda_{IQP})$')

ax.set_ylabel('Test score')

ax.set_title('IQP kernel test score')

####
# plt.subplots_adjust(hspace=-0.6,wspace=0.4)
fig.tight_layout()
#fig.savefig(f'{save_folder}{dataset_name}_geometric_difference_IQP_rbf.pdf')